In [1]:
import pandas as pd

In [2]:
# Standardizing values
def state(x):
    if 'A & n islands' == x or 'A and n islands' == x:
        return 'Andaman and nicobar'
    elif 'D & n haveli' == x or 'D&n haveli' == x or 'Dandn haveli' == x:
        return 'Dadra and nagar haveli'
    elif 'Daman & diu' == x:
        return 'Daman and diu'
    elif 'Delhi (ut)' == x or 'Delhi ut' == x:
        return 'Delhi'
    elif 'Jammu & kashmir' == x:
        return 'Jammu and kashmir'
    else:
        return x


# This column will represent administrative type of the region (state/Union territory)
ut = ['Andaman and nicobar', 'Chandigarh', 'Dadra and nagar haveli', 'Daman and diu', 'Delhi', 'Jammu and kashmir', 'Puducherry', 'Lakshadweep']


def administration(x):
    if x in ut:
        return 'UT'
    else:
        return 'State'


# Standard column order for all the categories
col_order = [0, -1, 1, 2, 3, 4, 5]

# Educational status of the victims

In [3]:
edu_decade = pd.read_csv(r".\Raw data\Educational_status.csv")
edu_2013 = pd.read_csv(r".\Raw data\Educational_status_2013.csv")
edu_2014 = pd.read_csv(r".\Raw data\Educational_status_2014.csv")

# Removing unwanted column
# Since transgender data only avaliable for a year(2014), we are going to neglect transgender data.
edu_2014["New Total"] = edu_2014.Total-edu_2014.Transgender

del edu_2014["Transgender"]
del edu_2014["Total"]

# Standardizing column name accross dataframes
edu_col = ["state/ut", "year", "educational status", "male", "female", "total"]

edu_decade.columns = edu_col
edu_2013.columns = edu_col
edu_2014.columns = edu_col

# Combining dataframes and resetting index
edu = pd.concat([edu_decade, edu_2013, edu_2014]).reset_index(drop=True)

# Cleaning "state/ut" column
edu["state/ut"] = edu["state/ut"].apply([lambda x: x.capitalize()])
edu["state/ut"] = edu["state/ut"].apply(lambda x: state(x))
edu = edu[~edu['state/ut'].isin(['Total (all india)', 'Total (states)', 'Total (uts)'])]


# Cleaning "educational status" column
# Standardizing values
def edu_status(x):
    if 'Diploma/ITI/Certificate' == x:
        return 'Diploma'
    elif 'Graduate and above' == x:
        return 'Graduate'
    elif 'Hr. Secondary/Intermediate/Pre-University' == x or 'Higher Secondary/ Intermediate/ Pre-University (upto class-12)' == x or 'Hr. Secondary/Intermediate/Pre-Universit'== x:
        return 'Hr. Secondary'
    elif 'Matriculate/Secondary (upto class-10)' == x or 'Matriculate/Secondary' == x:
        return 'Secondary'
    elif 'Middle (upto class-8)' == x:
        return 'Middle'
    elif 'Primary (upto class-5)' == x:
        return 'Primary'
    elif 'Professionals (MBA; etc.)' == x:
        return 'Post Graduate and Above'
    else:
        return x


edu['educational status'] = edu['educational status'].apply(lambda x: edu_status(x))
edu = edu[edu['educational status'] != "Total"]

# Creating new field
edu = edu.assign(administration=edu['state/ut'].apply(lambda x: administration(x)))

# Rearranging, renaming columns and sorting values 
edu = edu[edu.columns[col_order]]
edu = edu.sort_values(['state/ut', 'year'], ascending=[True, True]).reset_index(drop=True)
edu.columns = ['State/UT', 'Administration', 'Year', 'Educational status', 'Male', 'Female', 'Total']

# Cause

In [4]:
cause_decade = pd.read_csv(r".\Raw data\causes.csv")
cause_2013 = pd.read_csv(r".\Raw data\causes_2013.csv")
cause_2014 = pd.read_csv(r".\Raw data\Causes_2014.csv", encoding='latin1')

# Keeping only the required columns
cause_1 = cause_decade[cause_decade.columns[cause_decade.columns.isin(['STATE/UT', 'Year', 'CAUSE', 'Total Male', 'Total Female', 'Grand Total'])]]
cause_2 = cause_2013[cause_2013.columns[cause_2013.columns.isin(['STATE/UT', 'Year', 'CAUSE', 'Total Male', 'Total Female', 'Grand Total'])]]
cause_3 = cause_2014[cause_2014.columns[cause_2014.columns.isin(['States/UTs', 'Year', 'CrimeHead', 'Male_Total', 'Female_Total', 'Transgender_Total', 'Grand Total'])]]

# Removing unwanted columns
cause_3 = cause_3.assign(Total=cause_3.loc[:, 'Grand Total']-cause_3.loc[:, 'Transgender_Total'])
cause_3.drop(['Transgender_Total', 'Grand Total'], axis=1, inplace=True)

# Standardizing column's name
cause_cols = ['state/ut', 'year', 'cause', 'male', 'female', 'total']
cause_1.columns = cause_cols
cause_2.columns = cause_cols
cause_3.columns = cause_cols

# Concatenating DataFrames Filling in null values and formatting datatype
cause = pd.concat([cause_1, cause_2, cause_3]).reset_index(drop=True)
cause['total'].fillna(0, inplace=True)
cause['total'] = cause['total'].astype('int64')

# Cleaning 'State/UT' column
cause['state/ut'] = cause['state/ut'].apply(lambda x: x.capitalize())
cause['state/ut'] = cause['state/ut'].apply(lambda x: state(x))
cause = cause[~cause['state/ut'].isin(['Total (all india)', 'Total (states)', 'Total (uts)'])]


# Cleaning 'Causes' column
def causes(x):
    if x == 'Bankruptcy or Sudden change in Economic' or x == 'Bankruptcy or indebtedness' or x == 'Bankruptcy or Sudden change in Economic Status':
        return 'Bankruptcy'
    elif x == 'Death of Dear Person' or x == 'Death of dear person':
        return 'Death of Dear Person'
    elif x == 'Dowry Related Issues':
        return 'Dowry Dispute'
    elif x == 'Drug abuse/addiction':
        return 'Drug Abuse/Addiction'
    elif x == 'Fall in social reputation':
        return 'Fall in Social Reputation'
    elif x == 'Ideological causes/Hero worshipping':
        return 'Ideological Causes/Hero Worshipping'
    elif x == 'Illness (Aids/STD)':
        return 'AIDS/STD'
    elif x == 'Insanity/Mental illness':
        return 'Insanity/Mental Illness'
    elif x == 'Love affairs':
        return 'Love Affairs'
    elif x == 'Not having Children (Barrenness/Impotency' or x == 'Not having Children(Barrenness/Impotency' :
        return 'Impotency/Infertility'
    elif x == 'Other prolonged illness' or x == 'Other Prolonged Illness':
        return 'Prolonged Illness'
    elif x == 'Other Causes (Please Specity)' or x == 'Other causes':
        return 'Others'
    elif x == 'Causes Not known' or x == 'Causes not known':
        return 'Others'
    elif x == 'Physical Abuse (Rape,Incest Etc.)' or x == 'Physical Abuse (Rape; etc.)' :
        return 'Physical Abuse (Rape,Incest Etc.)'
    elif x == 'Property dispute':
        return 'Property Dispute'
    elif x == 'Extra Marital affairs' or x == 'Suspected/Illicit relation':
        return 'Suspected/Illicit Relation'
    elif x == 'Other Family problems' or x == 'Marriage Related Issues':
        return 'Family Problems'
    elif x == 'Non Settlement of Marriage':
        return 'Cancellation/Non-Settlement of Marriage'
    else:
        return x


cause['cause'] = cause['cause'].apply(lambda x: causes(x))

# Removing unwanted values
cause = cause[~cause['cause'].isin(['Total', 'Total Illness', 'Illness'])]

# Creating New column, rearranging, renaming columns, sorting values and resetting index
cause = cause.assign(administration = cause['state/ut'].apply(lambda x: administration(x)))
cause = cause[cause.columns[col_order]]
cause = cause.sort_values(['state/ut', 'year'], ascending=[True, True]).reset_index(drop=True)
cause.columns = ['State/UT', 'Administration', 'Year', 'Cause', 'Male', 'Female', 'Total']

# Means adopted by victims

**For this category we ignore year 2014 data. Since, the data categorization in "Means adopted" field in 2014 data is vastly different from rest of the data ("Means adopted", 2001-2013)**

In [5]:
means_decade = pd.read_csv(r'.\Raw data\Means_adopted.csv')
means_2013 = pd.read_csv(r'.\Raw data\Means_adopted_2013.csv')

# Keeping required columns and Standardizing column's name
means_1 = means_decade[means_decade.columns[means_decade.columns.isin(['STATE/UT', 'Year', 'CAUSE', 'Total Male', 'Total Female', 'Grand Total'])]]
means_2 = means_2013[means_2013.columns[means_2013.columns.isin(['STATE/UT', 'Year', 'CAUSE', 'Total Male', 'Total Female', 'Grand Total'])]]

means_cols = ['state/ut', 'year', 'means', 'male', 'female', 'total']
means_1.columns = means_cols
means_2.columns = means_cols

# Concatenating DataFrames
means = pd.concat([means_1, means_2]).reset_index(drop=True)

# Cleaning 'State/UT' column
means['state/ut'] = means['state/ut'].apply(lambda x: x.capitalize())
means['state/ut'] = means['state/ut'].apply(lambda x: state(x))
means = means[~means['state/ut'].isin(['Total (all india)', 'Total (states)', 'Total (uts)'])]


# Cleaning 'Means' column
def mean(x):
    if x == 'By coming under running vehicles/trains':
        return 'By Coming under running vehicles/trains'
    elif x == 'By Consuming other Poison':
        return 'By Consuming Other Poison'
    elif x == 'By Fire-Arms':
        return 'By Firearms'
    elif x == 'By Jumping off moving trains/vehicles':
        return 'By Jumping off Moving Vehicles/Trains'
    elif x == 'By Other means (please specify)' or x == 'By Jumping from':
        return 'By Other means'
    elif x == 'By Self inflicting  Injury':
        return 'By Self Infliction of injury'
    elif x == 'By consuming Sleeping Pills' or x == 'By Overdose of sleeping pills':
        return 'By Consuming Sleeping Pills'
    elif x == 'By touching electric wires':
        return 'By touching electric wire' 
    elif x == 'Building':
        return 'By Jumping from (Building)'
    elif x == 'Other sites':
        return 'By Jumping from (Other sites)'
    else:
        return x


means['means'] = means['means'].apply(lambda x: mean(x))
means = means[~means['means'].isin(['Total', 'Total (Jumping)', 'Total (Poison)'])]

# Creating New column
means = means.assign(administration = means['state/ut'].apply(lambda x: administration(x)))

# Rearranging and renaming columns, sorting values and resetting index
means = means[means.columns[col_order]]
means = means.sort_values(['state/ut', 'year'], ascending=[True, True]).reset_index(drop=True)
means.columns = ['State/UT', 'Administration', 'Year', 'Means adopted', 'Male', 'Female', 'Total']

# Marital status

In [6]:
marital_decade = pd.read_csv(r'.\Raw data\Social_status.csv')
marital_2013 = pd.read_csv(r'.\Raw data\Social_status_2013.csv')
marital_2014 = pd.read_csv(r'.\Raw data\Social_status_2014.csv', encoding='latin1')

# Keeping required columns
marital_1 = marital_decade[marital_decade.columns[marital_decade.columns.isin(['STATE/UT', 'Year', 'CAUSE', 'Male', 'Female', 'Total'])]]
marital_2 = marital_2013[marital_2013.columns[marital_2013.columns.isin(['STATE/UT', 'Year', 'CAUSE', 'Male', 'Female', 'Total'])]]
marital_3 = marital_2014[marital_2014.columns[marital_2014.columns.isin(['States/UTs', 'Year', 'CrimeHead', 'Male', 'Female', 'Transgender', 'Total'])]]

# Removing unwanted columns
marital_3 = marital_3.assign(Totals=marital_3.loc[:, 'Total']-marital_3.loc[:, 'Transgender'])
marital_3.drop(['Transgender', 'Total'], axis=1, inplace=True)

# Standardizing column's name
marital_cols = ['state/ut', 'year', 'marital', 'male', 'female', 'total']

marital_1.columns = marital_cols
marital_2.columns = marital_cols
marital_3.columns = marital_cols

# Concatenating DataFrames and resetting index
marital = pd.concat([marital_1, marital_2, marital_3]).reset_index(drop=True)

# Cleaning 'State/UT' column
marital['state/ut'] = marital['state/ut'].apply(lambda x: x.capitalize())
marital['state/ut'] = marital['state/ut'].apply(lambda x: state(x))
marital = marital[~marital['state/ut'].isin(['Total (all india)', 'Total (states)', 'Total (uts)'])]


# Cleaning 'Means' column
def marital_status(x):
    if x == 'Seperated':
        return 'Separated'
    elif x == 'Un-Married' or x == 'Never Married':
        return 'Unmarried'
    elif x == 'Status not known':
        return 'Others'
    else:
        return x


marital['marital'] = marital['marital'].apply(lambda x: marital_status(x))
marital = marital[~marital['marital'].isin(['Total'])]

# Creating New column
marital = marital.assign(administration = marital['state/ut'].apply(lambda x: administration(x)))

# Rearranging and renaming columns, sorting values and resetting index
marital = marital[marital.columns[col_order]]
marital = marital.sort_values(['state/ut', 'year'], ascending=[True, True]).reset_index(drop=True)
marital.columns = ['State/UT', 'Administration', 'Year', 'Marital status', 'Male', 'Female', 'Total']

# Professional profile

**For this category we ignore year 2014 data. Since, the data categorization in "Professional profile" field in 2014 data is vastly different from rest of the data ("Professional profile", 2001-2013)**

In [7]:
profession_decade = pd.read_csv(r'.\Raw data\Professional_status.csv')
profession_2013 = pd.read_csv(r'.\Raw data\Professional_status_2013.csv')

# Keeping required columns
profession_1 = profession_decade[profession_decade.columns[profession_decade.columns.isin(['STATE/UT', 'Year', 'CAUSE', 'Total Male', 'Total Female', 'Grand Total'])]]
profession_2 = profession_2013[profession_2013.columns[profession_2013.columns.isin(['STATE/UT', 'Year', 'CAUSE', 'Total Male', 'Total Female', 'Grand Total'])]]

# Standardizing column's name
profession_cols = ['state/ut', 'year', 'profession', 'male', 'female', 'total']

profession_1.columns = profession_cols
profession_2.columns = profession_cols

# Concatenating DataFrames and resetting index
profession = pd.concat([profession_1, profession_2]).reset_index(drop=True)

# Cleaning 'State/UT' column
profession['state/ut'] = profession['state/ut'].apply(lambda x: x.capitalize())
profession['state/ut'] = profession['state/ut'].apply(lambda x: state(x))
profession = profession[~profession['state/ut'].isin(['Total (all india)', 'Total (states)', 'Total (uts)'])]


# Cleaning 'Causes' column
def professional(x):
    if 'Others (Please Specify)' == x:
        return 'Others'
    else:
        return x


profession['profession'] = profession['profession'].apply(lambda x: professional(x))
profession = profession[~profession['profession'].isin(['Total', 'Total Salaried', 'Total Self-employed'])]

# Creating New column
profession = profession.assign(administration = profession['state/ut'].apply(lambda x: administration(x)))

# Rearranging and renaming columns, sorting values and resetting index
profession = profession[profession.columns[col_order]]
profession = profession.sort_values(['state/ut', 'year'], ascending=[True, True]).reset_index(drop=True)
profession.columns = ['State/UT', 'Administration', 'Year', 'Professional profile', 'Male', 'Female', 'Total']

# Exporting data in excel file

In [8]:
edu.to_excel('Educational status.xlsx', index=False)
cause.to_excel('Circumstances.xlsx', index=False)
means.to_excel('Means adopted.xlsx', index=False)
marital.to_excel('Marital status.xlsx', index=False)
profession.to_excel('Professional profile.xlsx', index=False)